<a href="https://colab.research.google.com/github/cars1015/recsys_MyReserch/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Googleドライブをマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
!pip install bottleneck
import os
import bottleneck as bn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 KB 20.1 MB/s eta 0:00:00


In [ ]:
#データの選択
#data="netflix-prize"
data="ml-20m"
#data="msd"

dir = "/content/drive/MyDrive/recommend/" + data + "/pro_sg/"
pred=np.loadtxt(dir+"pred_te.txt")

In [ ]:

def NDCG(x_pred, x_test, k=100):
  
    user_num = x_pred.shape[0]

    idx_topk_part = bn.argpartition(-x_pred, k, axis=1)

    topk_part = x_pred[np.arange(user_num)[:, np.newaxis],
                       idx_topk_part[:, :k]]

    idx_part = np.argsort(-topk_part, axis=1)

    idx_topk = idx_topk_part[np.arange(user_num)[:, np.newaxis], idx_part]


    tp = 1. / np.log2(np.arange(2, k + 2))                       
    DCG = (x_test[np.arange(user_num)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in x_test.getnnz(axis=1)])
    return DCG / IDCG

def Recall(x_pred, x_test, k):
    users_num = x_pred.shape[0]

    idx = bn.argpartition(-x_pred, k, axis=1)
   
    X_pred_binary = np.zeros_like(x_pred, dtype=bool)
   
    X_pred_binary[np.arange(users_num)[:, np.newaxis], idx[:, :k]] = True
   
    X_true_binary = (x_test > 0).toarray()
   
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [ ]:
unique_sid = list()
with open(os.path.join(dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(dir, 'test_tr.csv'),
    os.path.join(dir, 'test_te.csv'))

batch_size_test=2000
N_test=test_data_tr.shape[0]
idx_list_test=range(N_test)
test=test_data_te[idx_list_test]

In [ ]:
n100_list,r20_list,r50_list=[],[],[]

for bnum,st_idx in enumerate(range(0,N_test,batch_size_test)):
  end_idx=min(st_idx+batch_size_test,N_test)
  X = test_data_tr[idx_list_test[st_idx:end_idx]]
  if sparse.isspmatrix(X):
        X = X.toarray()
        X = X.astype('float32')
  pred_val=pred[idx_list_test[st_idx:end_idx]]
  pred_val[X.nonzero()]=-np.inf
  n100_list.append(NDCG(pred_val, test[st_idx:end_idx], k=100))
  r20_list.append(Recall(pred_val, test[st_idx:end_idx], k=20))
  r50_list.append(Recall(pred_val, test[st_idx:end_idx], k=50))
n100_list = np.concatenate(n100_list)
r20_list = np.concatenate(r20_list)
r50_list = np.concatenate(r50_list)

In [ ]:

print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))

Test NDCG@100=0.42026 (0.00214)
Test Recall@20=0.39132 (0.00268)
Test Recall@50=0.52107 (0.00281)
